<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Position-space" data-toc-modified-id="Position-space-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Position space</a></span><ul class="toc-item"><li><span><a href="#Translation-of-COM-instead-of-rotation-around-origin" data-toc-modified-id="Translation-of-COM-instead-of-rotation-around-origin-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Translation of COM instead of rotation around origin</a></span></li><li><span><a href="#Determine-position-index" data-toc-modified-id="Determine-position-index-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Determine position index</a></span></li></ul></li><li><span><a href="#Determine-internal-coordinate-system-(principal-axes)" data-toc-modified-id="Determine-internal-coordinate-system-(principal-axes)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Determine internal coordinate system (principal axes)</a></span></li><li><span><a href="#Full-conversion-3N-coordinates-<->-7D-gridpoint" data-toc-modified-id="Full-conversion-3N-coordinates-<->-7D-gridpoint-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Full conversion 3N coordinates &lt;-&gt; 7D gridpoint</a></span></li></ul></div>

In [1]:
import sys
import os

try:
    os.chdir(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup/molgri")
    sys.path.append(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup")
except FileNotFoundError:
    os.chdir(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    sys.path.append(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    
import warnings
warnings.filterwarnings("ignore")

In [14]:
import numpy as np
from numpy.typing import NDArray
import MDAnalysis as mda
from scipy.spatial.distance import cdist
from MDAnalysis.analysis.base import AnalysisFromFunction

from molgri.plotting.widgets import ViewManager
from molgri.paths import PATH_OUTPUT_PT
from molgri.space.fullgrid import FullGrid

## Position space



### Translation of COM instead of rotation around origin

In [3]:
# generated a file with
# python -m molgri.scripts.generate_pt -m1 H2O -m2 H2O -o "12" -b "8" -t "linspace(0.2, 1, 5)" --recal

pt_name = "H2O_H2O_0482"
pt_universe = mda.Universe(f"{PATH_OUTPUT_PT}{pt_name}.gro",
                           f"{PATH_OUTPUT_PT}{pt_name}.xtc")
len_pt = len(pt_universe.trajectory)

vm = ViewManager(pt_universe)
# plot every 8th frame
# because we have 8 orientations, every 8th structure should be translated version of each other
every_eighth_0 = np.arange(0, len_pt, 8)
print(list(every_eighth_0))
vm.plot_frames_overlapping(every_eighth_0[20:30])

[0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248, 256, 264, 272, 280, 288, 296, 304, 312, 320, 328, 336, 344, 352, 360, 368, 376, 384, 392, 400, 408, 416, 424, 432, 440, 448, 456, 464, 472]


NGLWidget()

In [4]:
vm.fresh_view()
every_eighth_3 = np.arange(3, len_pt, 8)
print(list(every_eighth_3))
vm.plot_frames_overlapping(every_eighth_3[20:30])

[3, 11, 19, 27, 35, 43, 51, 59, 67, 75, 83, 91, 99, 107, 115, 123, 131, 139, 147, 155, 163, 171, 179, 187, 195, 203, 211, 219, 227, 235, 243, 251, 259, 267, 275, 283, 291, 299, 307, 315, 323, 331, 339, 347, 355, 363, 371, 379, 387, 395, 403, 411, 419, 427, 435, 443, 451, 459, 467, 475]


NGLWidget()

### Determine position index

In [21]:
def _assing_2_closest_point(points_trajectory, points_grid, metric):
    pass

def assign_2_position_grid(my_trajectory: mda.Universe, position_grid_points: NDArray, second_molecule: str) -> NDArray:
    """
    Assign every frame of the trajectory (or PT) to the best fitting point of position grid

    Returns:
        an array of position grid indices
    """
    # step 1: determine the radial layer index from norm of the COM
    norm_coms = AnalysisFromFunction(lambda ag: np.linalg.norm(ag.center_of_mass()),
                         my_trajectory.trajectory,
                         my_trajectory.select_atoms(second_molecule))
    norm_coms.run()
    norms_along_trajectory = norm_coms.results['timeseries']
    # find available grid radii
    coms_available = np.linalg.norm(position_grid_points, axis=1)
    grid_radii = np.unique(coms_available)
    # find closest available grid radius
    t_selection = AnalysisFromFunction(lambda ag: np.argmin(np.abs(grid_radii-np.linalg.norm(ag.center_of_mass())), axis=0),
                                     my_trajectory.trajectory,
                                     my_trajectory.select_atoms(second_molecule))
    t_selection.run()
    t_indices = t_selection.results['timeseries'].flatten()
    # step 2: now using a normalized com and a metric on a sphere, determine which of self.o_grid is closest
    o_grid_points = self.full_grid.position_grid.get_o_grid().get_grid_as_array()
    o_selection = AnalysisFromFunction(lambda ag: np.argmin(cdist(o_grid_points, normalise_vectors(
        ag.center_of_mass())[np.newaxis, :], metric="cos"), axis=0),
                                     self.trajectory_universe.trajectory,
                                     self.trajectory_universe.select_atoms(self.second_molecule_selection))
    o_selection.run()
    o_indices = o_selection.results['timeseries'].flatten()

    # step 3: sum up the layer index and o index correctly
    n_orientations = len(position_grid_points) // len(grid_radii)
    return np.array(layer_indices * n_orientations + o_indices, dtype=int)
    

my_full_grid = FullGrid(b_grid_name="8", o_grid_name="12", t_grid_name="linspace(0.2, 1, 5)")
my_position_grid = my_full_grid.get_position_grid_as_array()

assign_2_position_grid(pt_universe, my_position_grid, second_molecule)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [ ]:
    # Return the indices of the bins to which each value in input array belongs.
    # If larger than the last bin, last bin index will still be given
    norms_along_trajectory[norms_along_trajectory > vor_radii[-1]] = vor_radii[-1]
    layer_indices = np.digitize(norms_along_trajectory, vor_radii, right=True)

    

## Determine internal coordinate system (principal axes)

In [6]:
# determine principal axes

# consider only the moving molecule
second_molecule = "bynum 4:6"

# principal axes of the first frame
pa_zero = pt_universe.select_atoms(second_molecule).principal_axes()
com_zero = pt_universe.select_atoms(second_molecule).center_of_mass()
print(np.round(pa_zero, 3))
print(com_zero+pa_zero[0])

# display principal axes of first frame
vm.fresh_view()
vm.plot_ith_frame(0)
vm.add_principal_axes(com_zero, pa_zero)
vm.view



[[ 0.     1.     0.   ]
 [-0.599  0.    -0.8  ]
 [-0.8    0.     0.599]]
[-1.70130157  1.          1.05146214]


NGLWidget()

In [8]:
vm.fresh_view()

# principal axes change in relation to the stationary coordinate system when the molecule is re-oriented
# but they remain the same in relation to the molecule itself

# using some selected PT frames for demonstration
for i in [20, 63, 284]:
    pt_universe.trajectory[i]
    pa_current = pt_universe.select_atoms(second_molecule).principal_axes()
    com_current = pt_universe.select_atoms(second_molecule).center_of_mass()
    print(np.round(pa_current, 3))

    # display this
    vm.plot_ith_frame(i)
    vm.add_principal_axes(com_current, pa_current)
vm.view

[[ 0.     0.     1.   ]
 [ 0.8   -0.599  0.   ]
 [ 0.599  0.8    0.   ]]
[[ 0.     0.     1.   ]
 [ 0.8    0.599  0.   ]
 [-0.599  0.8    0.   ]]
[[ 0.     0.     1.   ]
 [ 0.8   -0.599  0.   ]
 [ 0.599  0.8    0.   ]]


NGLWidget()

## Full conversion 3N coordinates <-> 7D gridpoint

In [ ]:
# -> direction: INPUT start coordinates (3N), 7D gridpoint; OUTPUT end coordinates (3N)
# <- direction: INPUT end coordinates (3N), start coordinates (3N); OUTPUT 7D gridpoint